In [ ]:
# RUNNING FOR THE FIRST TIME: Load & cd into gdrive for persistent data
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/'My Drive'

# Download/Install/Configure CUDALucas
!wget https://raw.github.com/tdulcet/Linux-System-Information/master/info.sh -qO - | bash -s # Check System Info
!nvidia-smi --query-gpu=gpu_name --format=csv,noheader # Output what GPU is assigned to this Notebook
!wget https://raw.github.com/tdulcet/Distributed-Computing-Scripts/master/cudalucas.sh -q # download CUDALucas script
!sed -i '/^if ! COMPUTE=/,/^fi/!b; /^fi/a echo "$COMPUTE"' cudalucas.sh # Output CUDA compute capability of GPU
!sed -i 's/\/$COMPUTE/\/--generate-code arch=compute_35,code=sm_35 --generate-code arch=compute_50,code=sm_50 --generate-code arch=compute_60,code=sm_60 --generate-code arch=compute_70,code=sm_70 --generate-code arch=compute_75,code=sm_75/' cudalucas.sh
!sed -i '/^GPU=/,/^fi/ s/^/# /' cudalucas.sh # Do not check for an Nvidia GPU
!sed -i '/^nohup / s/^/# /' cudalucas.sh # Do not start PrimeNet
!sed -i '/^crontab / s/^/# /' cudalucas.sh # Do not create a cronjob
!bash -- cudalucas.sh "" # Do not pass any additional parameters here

# Download/Install/Configure Prime95
!wget https://raw.github.com/tdulcet/Distributed-Computing-Scripts/master/mprime.sh -q # Download Prime95
!rm mprime.sh # remove any old download of the Prime95 script (for case of using 2 notebook maximum)
!sed -i '/^nohup / s/^/# /' mprime.sh # Do not start Prime95
!sed -i '/^crontab / s/^/# /' mprime.sh # Do not create a cronjob
!sed -i 's/"mprime"/"mprime_gpu"/' mprime.sh # Name the folder specific to the runtime type
!bash -- mprime.sh psu # Replace 'psu' with your own username if you wish and, optionally, add a name for the machine

# Run CUDALucas and Mprime
!cd cudalucas; nohup python primenet.py -d -T 100 -u psu -p vikings & # Start PrimeNet to get/submit work. If you wish, replace 'psu' & 'vikings' with your GIMPS Username & Password
!sleep 1.5
!cd cudalucas && chmod 777 CUDALucas; nohup ./CUDALucas & # Outputs to nohup.out in cudalucas directory
!cd mprime_gpu && chmod 777 mprime && ./mprime -d

In [ ]:
# RUNNING SUBSEQUENT TIMES: Restarting CUDALucas when Colab/you shut the process/notebook down
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/'My Drive'/

# Run CUDALucas and Mprime
!cd cudalucas; nohup python primenet.py -d -T 100 -u psu -p vikings & # Start PrimeNet to get/submit work. If you wish, replace 'psu' & 'vikings' with your GIMPS Username & Password
!sleep 1.5
!cd cudalucas && chmod 777 CUDALucas; nohup ./CUDALucas & # Outputs to nohup.out in cudalucas directory
!cd mprime_gpu && chmod 777 mprime && ./mprime -d

In [1]:
# DEBUGGING
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/'My Drive'/

!nvidia-smi --query-gpu=gpu_name --format=csv,noheader # Output what GPU is assigned to this notebook
!tail -n 100 cudalucas/nohup.out # view CUDALucas progress